In [1]:
import weave
weave.init("graphrag-poc")

weave version 0.51.23 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: vedmanivaidya.
View Weave data at https://wandb.ai/vedmanivaidya/graphrag-poc/weave


In [1]:
# Enable auto-reloading of modules
%load_ext autoreload
%autoreload 2

In [2]:
# import logging
 
# # Configure logging to write to a file, setting the level to INFO
# logging.basicConfig(level=logging.DEBUG,
#                     format='%(asctime)s - %(levelname)s - %(message)s',
#                     filename='app.log',  # Log messages are written to this file
#                     filemode='a')  # Append mode
 
# # Create a StreamHandler to output logs to the console as well
# console_handler = logging.StreamHandler()
# console_handler.setLevel(logging.DEBUG)  # Set the log level for the console
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# console_handler.setFormatter(formatter)
 
# # Add the console handler to the root logger
# logging.getLogger('').addHandler(console_handler)

In [3]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

from llama_index.core.schema import NodeWithScore, MetadataMode


In [4]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
openai_llm = AzureOpenAI(
    model=os.getenv("DEPLOYMENT_NAME"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0.0,
)
openai_embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    # deployment_name="my-custom-embedding",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
)
embeddings = openai_embed_model.get_text_embedding("Hello, world!")
len(embeddings)

1536

In [5]:
gemini_llm = Gemini(
    model="models/gemini-1.5-flash-latest",
    temperature=0.0,
)
gemini_embed_model = GeminiEmbedding()
embeddings = gemini_embed_model.get_text_embedding("Hello, world!")
len(embeddings)

768

In [6]:
from llama_index.core import Settings

Settings.llm = gemini_llm
Settings.embed_model = gemini_embed_model

In [7]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    "D:\projects\graphrag-poc\data\selection 1"
).load_data()

In [9]:
from graphrag_poc.custom_llama_index.custom_neo4j_property_graph import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="12345678",
    url="bolt://localhost:7687",
    database="graphrag-openai-custom-extractor-prompt-gemini",
    refresh_schema=False,
)

In [30]:
graph_store.get_llama_nodes(["16c29ab7-ddb1-4bd8-b875-6554ffc07a96"])

[TextNode(id_='16c29ab7-ddb1-4bd8-b875-6554ffc07a96', embedding=None, metadata={'file_path': "D:\\projects\\graphrag-poc\\data\\selection 1\\A Student's Guide to Startups (HIVE).txt", 'file_name': "A Student's Guide to Startups (HIVE).txt", 'file_type': 'text/plain', 'file_size': 35856, 'creation_date': '2024-11-26', 'last_modified_date': '2024-11-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='89726fb7-4d79-4142-8bd4-c43cac27e169', node_type='4', metadata={'file_path': "D:\\projects\\graphrag-poc\\data\\selection 1\\A Student's Guide to Startups (HIVE).txt", 'file_name': "A Student's Guide to Startups (HIVE).txt", 'file_type': 'text/plain', 'file_size': 35856, 'creation_date': '2024-11-26', 'last_mod

In [10]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=50,
)
nodes = splitter.get_nodes_from_documents(documents)
len(nodes)
# create a list of lists each of 10 nodes
nodes_list = [nodes[i : i + 10] for i in range(0, len(nodes), 10)]
print(len(nodes_list))

8


In [11]:
prompt_template = (
    "You are a knowledge graph expert specializing in extracting structured information from text about startup founders, companies, and entrepreneurship guidance, particularly from Paul Graham's writings."
    "\nYour task is to extract up to {max_knowledge_triplets} knowledge triplets from the provided text. "
    "\n If there are more triplets that can be extracted, then extract as many as needed to capture all the information"
    "A knowledge triplet consists of (head, relation, tail) along with their types and properties."
    "\n\nCONTEXT AWARENESS:"
    "\n- Focus on startup-related entities: founders, companies, investors, concepts"
    "\n- Identify key entrepreneurship principles and advice"
    "\n- Capture relationships between people, organizations, and ideas"
    "\n- Extract time-sensitive information when available (founded_date, funding_rounds, etc.)"
    "\n---------------------\n"
    "INITIAL ONTOLOGY:\n"
    "Entity Types: {allowed_entity_types}\n"
    "Entity Properties: {allowed_entity_properties}\n"
    "Relation Types: {allowed_relation_types}\n"
    "Relation Properties: {allowed_relation_properties}\n"
    "\n"
    "Use these types as a starting point, but introduce new types if necessary based on the context.\n"
    "If the Entity Properties, Relation Properties contain property 'description', generate a context aware detailed description, which will have some uniqe non generic information addition"
    "\n"
    "GUIDELINES:\n"
    "- Output in JSON format: [{{'head': '', 'head_type': '', 'head_props': {{...}}, 'relation': '', 'relation_props': {{...}}, 'tail': '', 'tail_type': '', 'tail_props': {{...}}}}]\n"
    "- Use the most complete form for entities (e.g., 'United States of America' instead of 'USA') but where its ambiguous, use the entity as it is\n"
    "- Keep entities concise\n"
    "- While writing description property for entities and relations keep context in mind and just dont write the description of the entity or relation, but the description of the entity or relation in the context of the text\n"
    "- Ensure the knowledge graph is coherent and easily understandable\n"
    "- While extracting relation, use singular form of the relation. Use EXPAND instead of EXPANDS or EXPECT instead of EXPECTS\n"
    "- The goal is to make relations as generics as possible, so that there are less duplicate relations in the graph, which have same meaning\n"
    "- If there are two names present in the text treat them as separate entities. For example Jessica Livingston and Robert Morris then they are two separate entities Jessica Livingston, Robert Morris\n"
    "- Focus on startup-specific metrics and relationships (funding rounds, valuations, mentor relationships)\n"
    "- Capture temporal aspects of relationships when mentioned (founding dates, acquisition dates)\n"
    "- Include relevant contextual properties (industry sector, technology stack, market focus)\n"
    "---------------------\n"
    "EXAMPLE:\n"
    "Text Input: \nTim Cook, CEO of Apple Inc., announced the new Apple Watch that monitors heart health. "
    "UC Berkeley researchers studied the benefits of apples.\n"
    "Example Output:\n"
    "[{{'head': 'Tim Cook', 'head_type': 'PERSON', 'head_props': {{'description': 'Technology executive who made the product announcement for Apple Watch, demonstrating leadership in health-focused technology initiatives'}}, 'relation': 'CEO_OF', 'relation_props': {{'description': 'Executive leadership role involving product announcements and strategic health technology initiatives'}}, 'tail': 'Apple Inc.', 'tail_type': 'COMPANY', 'tail_props': {{'description': 'Technology company expanding into health monitoring through wearable devices'}}}},\n"
    " {{'head': 'Apple Inc.', 'head_type': 'COMPANY', 'head_props': {{'description': 'Company developing health-focused consumer technology products under Tim Cook's leadership'}}, 'relation': 'PRODUCE', 'relation_props': {{'description': 'Strategic initiative to enter health monitoring market through consumer devices'}}, 'tail': 'Apple Watch', 'tail_type': 'PRODUCT', 'tail_props': {{'description': 'Health-focused smartwatch representing Apple's expansion into medical monitoring technology'}}}},\n"
    " {{'head': 'Apple Watch', 'head_type': 'PRODUCT', 'head_props': {{'description': 'Wearable device specifically designed to track and monitor user health metrics'}}, 'relation': 'MONITOR', 'relation_props': {{'description': 'Continuous health monitoring capability focusing on cardiac metrics'}}, 'tail': 'heart health', 'tail_type': 'HEALTH_METRIC', 'tail_props': {{'description': 'Critical health metric monitored through Apple Watch's advanced sensors'}}}},\n"
    " {{'head': 'UC Berkeley', 'head_type': 'UNIVERSITY', 'head_props': {{'description': 'Academic institution conducting research on nutritional benefits and health impacts'}}, 'relation': 'STUDY', 'relation_props': {{'description': 'Academic research focusing on health benefits and nutritional value analysis'}}, 'tail': 'benefits of apples', 'tail_type': 'RESEARCH_TOPIC', 'tail_props': {{'description': 'Scientific investigation into the health advantages and nutritional properties of apples'}}}}]\n"
    "---------------------\n"
    "MAKE SURE TO FOLLOW THE EXAMPLE FORMAT STRICTLY FOR EACH KNOWLEDGE TRIPLET AND GIVE THE OUTPUT IN JSON LIST FORMAT ONLY AS ITS PARSED USING CODE AND IT NEEDS TO MATCH THE EXAMPLE FORMAT STRICTLY"
    "Text: {text}\n"
    "Output:\n"
)

In [12]:
from typing import Any, Dict, List, Tuple
import re
import json
from llama_index.core.graph_stores.types import (
    EntityNode,
    Relation,
)

def parse_dynamic_triplets_with_props(
    llm_output: str,
) -> List[Tuple[EntityNode, Relation, EntityNode]]:
    """
    Parse the LLM output and convert it into a list of entity-relation-entity triplets.
    This function is flexible and can handle various output formats.

    Args:
        llm_output (str): The output from the LLM, which may be JSON-like or plain text.

    Returns:
        List[Tuple[EntityNode, Relation, EntityNode]]: A list of triplets.
    """
    triplets = []

    try:
        #first extract the json from markdown
        llm_output = re.search(r'```json\s*([\s\S]*)\s*```', llm_output).group(1)
        # Attempt to parse the output as JSON
        data = json.loads(llm_output)
        for item in data:
            head = item.get("head")
            head_type = item.get("head_type")
            head_props = item.get("head_props", {})
            relation = item.get("relation")
            relation_props = item.get("relation_props", {})
            tail = item.get("tail")
            tail_type = item.get("tail_type")
            tail_props = item.get("tail_props", {})

            if head and head_type and relation and tail and tail_type:
                head_node = EntityNode(
                    name=head, label=head_type, properties=head_props
                )
                tail_node = EntityNode(
                    name=tail, label=tail_type, properties=tail_props
                )
                relation_node = Relation(
                    source_id=head_node.id,
                    target_id=tail_node.id,
                    label=relation,
                    properties=relation_props,
                )
                triplets.append((head_node, relation_node, tail_node))
    except json.JSONDecodeError:
        # Flexible pattern to match the key-value pairs for head, head_type, head_props, relation, relation_props, tail, tail_type, and tail_props
        pattern = r'[\{"\']head[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']head_type[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']head_props[\}"\']\s*:\s*\{(.*?)\}\s*,\s*[\{"\']relation[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']relation_props[\}"\']\s*:\s*\{(.*?)\}\s*,\s*[\{"\']tail[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']tail_type[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']tail_props[\}"\']\s*:\s*\{(.*?)\}\s*'

        # Find all matches in the output
        matches = re.findall(pattern, llm_output)

        for match in matches:
            (
                head,
                head_type,
                head_props,
                relation,
                relation_props,
                tail,
                tail_type,
                tail_props,
            ) = match

            # Use more robust parsing for properties
            def parse_props(props_str: str) -> Dict[str, Any]:
                try:
                    # Handle mixed quotes and convert to a proper dictionary
                    props_str = props_str.replace("'", '"')
                    return json.loads(f"{{{props_str}}}")
                except json.JSONDecodeError:
                    return {}

            head_props_dict = parse_props(head_props)
            relation_props_dict = parse_props(relation_props)
            tail_props_dict = parse_props(tail_props)

            head_node = EntityNode(
                name=head, label=head_type, properties=head_props_dict
            )
            tail_node = EntityNode(
                name=tail, label=tail_type, properties=tail_props_dict
            )
            relation_node = Relation(
                source_id=head_node.id,
                target_id=tail_node.id,
                label=relation,
                properties=relation_props_dict,
            )
            triplets.append((head_node, relation_node, tail_node))
    return triplets

In [13]:
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor

dyn_llm_path_extractor = DynamicLLMPathExtractor(
    llm=Settings.llm,
    allowed_entity_props=["description"],
    allowed_relation_props=["description"],
    extract_prompt=prompt_template,
    parse_fn=parse_dynamic_triplets_with_props,
)

In [15]:
# from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
# neo4j_vector = Neo4jVectorStore(
#     username="neo4j",
#     password="12345678",
#     url="bolt://localhost:7687",
#     database="graphrag-openai-custom-extractor-prompt-gemini",
#     hybrid_search = True,
#     embedding_dimension=768
# )


In [14]:
from llama_index.core import PropertyGraphIndex
# from llama_index.core.indices.property_graph import SimpleLLMPathExtractor

# index = PropertyGraphIndex(
#     nodes_list[0],
#     embed_model=Settings.embed_model,
#     kg_extractors=[dyn_llm_path_extractor],
#     property_graph_store=graph_store,
#     show_progress=True,
# )

index = PropertyGraphIndex.from_existing(
    # vector_store=neo4j_vector,
    property_graph_store=graph_store,
    llm=Settings.llm,
    embed_model=Settings.embed_model,
    kg_extractors=[dyn_llm_path_extractor],
    show_progress=True,
)

In [17]:
# index._insert_nodes(nodes_list[1])

In [18]:
# index._insert_nodes(nodes_list[2])

In [19]:
# import time
# index._insert_nodes(nodes_list[3])
# time.sleep(60)
# index._insert_nodes(nodes_list[4])
# time.sleep(60)
# index._insert_nodes(nodes_list[5])
# time.sleep(60)
# index._insert_nodes(nodes_list[6])
# time.sleep(60)
# index._insert_nodes(nodes_list[7])

In [20]:
# graph_store.structured_query("""
# CREATE VECTOR INDEX entity IF NOT EXISTS
# FOR (m:`__Entity__`)
# ON m.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 1536,
#  `vector.similarity_function`: 'cosine'
# }}
# """)

In [21]:
# similarity_threshold = 0.9
# word_edit_distance = 4
# data = graph_store.structured_query("""
# MATCH (e:__Entity__)
# CALL {
#   WITH e
#   CALL db.index.vector.queryNodes('entity', 10, e.embedding)
#   YIELD node, score
#   WITH node, score
#   WHERE score > toFLoat($cutoff)
#       AND (toLower(node.name) CONTAINS toLower(e.name) OR toLower(e.name) CONTAINS toLower(node.name)
#            OR apoc.text.distance(toLower(node.name), toLower(e.name)) < $distance)
#       AND labels(e) = labels(node)
#   WITH node, score
#   ORDER BY node.name
#   RETURN collect(node) AS nodes
# }
# WITH distinct nodes
# WHERE size(nodes) > 1
# WITH collect([n in nodes | n.name]) AS results
# UNWIND range(0, size(results)-1, 1) as index
# WITH results, index, results[index] as result
# WITH apoc.coll.sort(reduce(acc = result, index2 IN range(0, size(results)-1, 1) |
#         CASE WHEN index <> index2 AND
#             size(apoc.coll.intersection(acc, results[index2])) > 0
#             THEN apoc.coll.union(acc, results[index2])
#             ELSE acc
#         END
# )) as combinedResult
# WITH distinct(combinedResult) as combinedResult
# // extra filtering
# WITH collect(combinedResult) as allCombinedResults
# UNWIND range(0, size(allCombinedResults)-1, 1) as combinedResultIndex
# WITH allCombinedResults[combinedResultIndex] as combinedResult, combinedResultIndex, allCombinedResults
# WHERE NOT any(x IN range(0,size(allCombinedResults)-1,1) 
#     WHERE x <> combinedResultIndex
#     AND apoc.coll.containsAll(allCombinedResults[x], combinedResult)
# )
# RETURN combinedResult  
# """, param_map={'cutoff': similarity_threshold, 'distance': word_edit_distance})
# for row in data:
#     print(row)

In [15]:
from llama_index.core import PromptTemplate

prompt_template = """
You are an expert qna assistant tasked with providing comprehensive, well-structured answers based SOLELY on the given context. Your role is to synthesize and explain information from the provided context without adding external knowledge or making assumptions.
 
1. Context Adherence:
- ONLY use information that is explicitly present in the provided context
- Do not add external knowledge, even if you know it's correct
- If the context is insufficient to fully answer the question, acknowledge the limitations of available information
- Do not make assumptions or inferences beyond what's directly supported by the context
- If certain aspects of the question cannot be answered from the context, clearly state this rather than making up information
 
2. Answer Structure:
- Break down the answer into clear, numbered points when appropriate
- Use bold formatting for key concepts using **text**
- Maintain a logical flow from one point to the next
- End with a brief summary that ties the main points together
 
3. Content Development:
- Expand on the context provided while staying strictly within its boundaries
- Provide detailed explanations for each major point using only information from the context
- Include relevant examples or implications only if present in the context
- Connect related concepts to show relationships and broader impact
- Ensure each point is substantive and supported by the source material
 
4. Language and Tone:
- Use clear, professional language
- Maintain an authoritative but accessible tone
- Define technical terms when they appear in the context
- Be precise and specific in explanations
- Use active voice when possible
 
5. Quality Control:
- Verify every statement against the provided context
- If the context is ambiguous or unclear, reflect this uncertainty in your answer
- When multiple sources discuss the same point, synthesize them coherently
- Address only those aspects of the question that can be answered from the given context
 
Remember: Your primary responsibility is to provide accurate responses based EXCLUSIVELY on the provided context. Never supplement the answer with external knowledge or assumptions, even if they would make the answer more complete.

Context information is provided below.
---------------------
{context_str}
---------------------
Please answer the following question based on this context: {query_str}
---------------------
Answer:
"""

custom_prompt = PromptTemplate(prompt_template)


In [24]:
from llama_index.core.indices.property_graph import (
    LLMSynonymRetriever,
    VectorContextRetriever
)

from llama_index.core.vector_stores.types import VectorStoreQueryMode

from graphrag_poc.custom_llama_index.custom_vector_retriever import CustomVectorContextRetriever

# vector_retriever = VectorContextRetriever(
#      graph_store,
#      embed_model=Settings.embed_model,
#     #  similarity_top_k=4,
#      path_depth=3,
#      include_text=True,
#      similarity_score=0.7,
#      limit=10,
#      similarity_top_k=10,
# )

llm_synonym_retriever = LLMSynonymRetriever(
    graph_store=graph_store,
    include_text=True,
    # include_properties=True,
    max_keywords=20,
    llm=Settings.llm,
)

custom_vector_retriever = CustomVectorContextRetriever(
     graph_store,
     embed_model=Settings.embed_model,
     path_depth=3,
     include_text=True,
     similarity_score=0.7,
     limit=100,
     similarity_top_k=10,
     mode=VectorStoreQueryMode.HYBRID,
     hybrid_top_k=10,
    #  alpha=0.75
)

In [25]:
# from llama_index.core.schema import QueryBundle
# query_bundle = QueryBundle(
#     query_str="What are the specific advantages of the 'grad student' model for startups, and why is it preferable to the 'law firm' model? It is okay if the answer is lengthy. Its is very critical that the answers are written using information available in the provided data."
# )
query_engine = index.as_query_engine(
    sub_retrievers=[custom_vector_retriever, llm_synonym_retriever],
    text_qa_template=custom_prompt,
    response_mode="compact",
    # # include_text=True,
    # # include_properties=True,
    # similarity_top_k=10,
    # limit=100,
    # similarity_score=0.75,
)
response:List[NodeWithScore] = query_engine.query(
    "What were the main reasons for the decline in inherited wealth as the primary source of riches among the Forbes 400 between 1982 and 2020?"
)
print(response)

The provided text states that the decrease in the percentage of inherited wealth among the richest Americans from 1982 to 2020 is not due to increased inheritance taxes; in fact, these taxes decreased significantly during that period.  Instead, the decline is attributed to the rise of individuals creating their own wealth.  This increase in self-made fortunes is primarily driven by two factors:

1. **Company Founding:**  A significant portion (roughly 3/4) of new fortunes in 2020 originated from starting companies, with 56 out of 73 new fortunes stemming from founders' or early employees' equity.  This contrasts sharply with 1982, where inheritance was the dominant source of wealth.

2. **Investment Management:** Another source of new wealth in 2020 was managing investment funds, accounting for 17 out of 73 new fortunes.  This was absent among the top 100 richest in 1982, indicating a shift in wealth creation strategies.

In summary, the decrease in inherited wealth among the Forbes 40

In [26]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='2be3170b-8cca-4be4-b1b1-235405b805b6', embedding=None, metadata={'file_path': 'D:\\projects\\graphrag-poc\\data\\selection 1\\How People Get Rich Now (HIVE).txt', 'file_name': 'How People Get Rich Now (HIVE).txt', 'file_type': 'text/plain', 'file_size': 14641, 'creation_date': '2024-11-26', 'last_modified_date': '2024-11-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5e19c9e8-b104-4193-8eb9-470fcc513b78', node_type='4', metadata={'file_path': 'D:\\projects\\graphrag-poc\\data\\selection 1\\How People Get Rich Now (HIVE).txt', 'file_name': 'How People Get Rich Now (HIVE).txt', 'file_type': 'text/plain', 'file_size': 14641, 'creation_date': '2024-11-26', 'last_modified

In [22]:
len(response.source_nodes)

2

In [66]:
# from pydantic import BaseModel
# class MyResponseNodes(BaseModel):
#     response_nodes: List[NodeWithScore]

In [67]:
# #create a MyResponseNodes from response.nodes
# my_response_nodes = MyResponseNodes(response_nodes=response.source_nodes)
# print(my_response_nodes.model_dump_json())

{"response_nodes":[{"node":{"id_":"2be3170b-8cca-4be4-b1b1-235405b805b6","embedding":null,"metadata":{"file_path":"D:\\projects\\graphrag-poc\\data\\selection 1\\How People Get Rich Now (HIVE).txt","file_name":"How People Get Rich Now (HIVE).txt","file_type":"text/plain","file_size":14641,"creation_date":"2024-11-26","last_modified_date":"2024-11-20"},"excluded_embed_metadata_keys":["file_name","file_type","file_size","creation_date","last_modified_date","last_accessed_date"],"excluded_llm_metadata_keys":["file_name","file_type","file_size","creation_date","last_modified_date","last_accessed_date"],"relationships":{"1":{"node_id":"5e19c9e8-b104-4193-8eb9-470fcc513b78","node_type":"4","metadata":{"file_path":"D:\\projects\\graphrag-poc\\data\\selection 1\\How People Get Rich Now (HIVE).txt","file_name":"How People Get Rich Now (HIVE).txt","file_type":"text/plain","file_size":14641,"creation_date":"2024-11-26","last_modified_date":"2024-11-20"},"hash":"638a3fc276bd116c3f2338f93ccc6b47968

In [27]:
from llama_index.core.schema import NodeWithScore, MetadataMode

response_nodes: List[NodeWithScore] = response.source_nodes

for node in response_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))
    print("--------------------------------")


file_path: D:\projects\graphrag-poc\data\selection 1\How People Get Rich Now (HIVE).txt

Here are some facts extracted from the provided text:

Inheritance -> SOURCE_OF -> Wealth of Richest Americans (1982)
Inheritance -> SOURCE_OF -> Wealth of Richest Americans (1982)
Oil and Real Estate -> SOURCE_OF -> New Fortunes (1982)
Oil and Real Estate -> SOURCE_OF -> New Fortunes (1982)
Forbes magazine -> PUBLISH -> list of richest Americans
Forbes magazine -> PUBLISH -> list of richest Americans
Company Founding -> SOURCE_OF -> Wealth of Richest Americans (2020)
Company Founding -> SOURCE_OF -> Wealth of Richest Americans (2020)

April 2021 Every year since 1982, Forbes magazine has published a list of the richest Americans. If we compare the 100 richest people in 1982 to the 100 richest in 2020, we notice some big differences. In 1982 the most common source of wealth was inheritance. Of the 100 richest people, 60 inherited from an ancestor. There were 10 du Pont heirs alone. By 2020 the numb

In [29]:
from IPython.display import Markdown, display
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [30]:
prompts_dict = query_engine.get_prompts()

In [31]:
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template**Text:** 

You are an expert assistant tasked with providing comprehensive, well-structured answers based SOLELY on the given context. Your role is to synthesize and explain information from the provided context without adding external knowledge or making assumptions.
 
1. Context Adherence:
- ONLY use information that is explicitly present in the provided context
- Do not add external knowledge, even if you know it's correct
- If the context is insufficient to fully answer the question, acknowledge the limitations of available information
- Do not make assumptions or inferences beyond what's directly supported by the context
- If certain aspects of the question cannot be answered from the context, clearly state this rather than making up information
 
2. Answer Structure:
- Begin with a concise introductory paragraph that directly addresses the main question
- Break down the answer into clear, numbered points when appropriate
- Use bold formatting for key concepts using **text**
- Maintain a log

**Prompt Key**: response_synthesizer:refine_template**Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
